In [1]:
import pandas as pd
import numpy as np
import seaborn as sns
import matplotlib.pyplot as plt
import os
from helper import DateToOrdinal

from sklearn.pipeline import make_pipeline

from sklearn.model_selection import cross_val_score

import xgboost as xgb

from sklearn.model_selection import GridSearchCV

In [2]:
data_folder = "data"
filename = "train.csv"
df = pd.read_csv(os.path.join("..", data_folder, filename))
df['date'] = pd.to_datetime(df['date'])
test_cols_plus_y = ["orders", "warehouse", "date", "holiday_name", "holiday", "shops_closed", "winter_school_holidays", "school_holidays", "id"]
df = df[test_cols_plus_y]

In [3]:
# Drop categorical columns
df = df.drop(columns=["holiday_name", "id"])

In [4]:
# Helpful functions
def create_lagged_features(df_l, value, lags):
    for lag in range(1, lags + 1):
        df_l[f't-{lag}'] = df_l[value].shift(lag)
    df_l.dropna(inplace=True)
    return df_l

def add_cyclic_sin_cos_features(df_c, datecolumn = "date"):
    # Create sin and cos features for day of year
    df_c['dayofyear_sin'] = np.sin(2 * np.pi * df_c[datecolumn].dt.dayofyear/365.25)
    df_c['dayofyear_cos'] = np.cos(2 * np.pi * df_c[datecolumn].dt.dayofyear/365.25)

    # Create sin and cos features for day of week
    df_c['dayofweek_sin'] = np.sin(2 * np.pi * df_c[datecolumn].dt.dayofweek/7)
    df_c['dayofweek_cos'] = np.cos(2 * np.pi * df_c[datecolumn].dt.dayofweek/7)

    return df_c

In [5]:
def cross_val_xgb(data):

    result_list = []

    try:
        data = data.set_index("date")
    except:
        pass

    xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', seed=42)
    xgb_pipe = make_pipeline(xgb_regressor)

    warehouses = data["warehouse"].unique().tolist()

    for warehouse in warehouses:
        warehouse_data = data[data["warehouse"] == warehouse]

        warehouse_data_X = warehouse_data.drop(columns=["orders", "warehouse"])
        warehouse_data_y = warehouse_data["orders"]

        final_score = abs(cross_val_score(xgb_pipe, warehouse_data_X, warehouse_data_y, cv=5, scoring="neg_mean_absolute_percentage_error").mean())

        result_list.append(final_score)
        print(warehouse, final_score)

    print(np.mean(result_list))

In [26]:
def cross_val_modelX(data, model):

    result_list = []

    try:
        data = data.set_index("date")
    except:
        pass


    warehouses = data["warehouse"].unique().tolist()

    for warehouse in warehouses:
        warehouse_data = data[data["warehouse"] == warehouse]

        warehouse_data_X = warehouse_data.drop(columns=["orders", "warehouse"])
        warehouse_data_y = warehouse_data["orders"]

        final_score = abs(cross_val_score(model, warehouse_data_X, warehouse_data_y, cv=5, scoring="neg_mean_absolute_percentage_error").mean())

        result_list.append(final_score)
        print(warehouse, final_score)

    print(np.mean(result_list))

In [6]:
# just the raw data
cross_val_xgb(df.copy())

Prague_1 0.1174834586213414
Brno_1 0.10992147215999762
Prague_2 0.113091330328192
Prague_3 0.10547881008750096
Munich_1 0.43589209391092093
Frankfurt_1 0.14427350539515674
Budapest_1 0.11177190359460883
0.1625589391568169


In [7]:
subset_df = df[["orders", "date", "warehouse"]]
subset_df["daterange"] = list(range(len(df)))
cross_val_xgb(subset_df.copy())

/tmp/ipykernel_1454/242619256.py:2: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
  subset_df["daterange"] = list(range(len(df)))


Prague_1 0.11169047729247325
Brno_1 0.10001992053566486
Prague_2 0.10268778518754171
Prague_3 0.10411751055475424
Munich_1 0.42130250965416216
Frankfurt_1 0.12137356009985359
Budapest_1 0.08241259370727397
0.14908633671881769


In [8]:
df_lagged = create_lagged_features(df.copy(), "orders", lags=14)

In [9]:
# Added only lagged features
cross_val_xgb(df_lagged.copy())

Prague_1 0.057678079158496855
Brno_1 0.06578618893692048
Prague_2 0.05767736144717182
Prague_3 0.061710224056729344
Munich_1 0.27913605188143553
Frankfurt_1 0.07204650074063652
Budapest_1 0.04908571769782595
0.09187430341703094


In [10]:
df_lagged["daterange"] = list(range(len(df_lagged)))
# Added lagged + timesteps
cross_val_xgb(df_lagged.copy())

Prague_1 0.0621815801247265
Brno_1 0.06821432579170626
Prague_2 0.05715227720377325
Prague_3 0.05972169262773184
Munich_1 0.283946944688967
Frankfurt_1 0.0714673571764366
Budapest_1 0.05246677709661729
0.09359299352999409


In [11]:
# Added only cyclic features
df_sin_cos = add_cyclic_sin_cos_features(df.copy(), datecolumn = "date")
cross_val_xgb(df_sin_cos.copy())

Prague_1 0.10539913561320854
Brno_1 0.10434351070523891
Prague_2 0.09916021117940663
Prague_3 0.09431401999812061
Munich_1 0.44199247112450574
Frankfurt_1 0.18666164034583246
Budapest_1 0.11556753022901456
0.16391978845647534


In [17]:
# all together
final_df = add_cyclic_sin_cos_features(df.copy(), datecolumn = "date")
final_df = create_lagged_features(final_df.copy(), "orders", lags=14)
final_df["daterange"] = list(range(len(final_df)))
cross_val_xgb(final_df.copy())

Prague_1 0.05568215483599996
Brno_1 0.0611919237773398
Prague_2 0.050446994096569296
Prague_3 0.0550562290729386
Munich_1 0.24588095047251152
Frankfurt_1 0.07224886959306018
Budapest_1 0.03877232609210141
0.08275420684864583


In [20]:
from sklearn.preprocessing import StandardScaler

def scale_t_columns(df):
    # Identify t-columns
    t_columns = [col for col in df.columns if col.startswith('t-') and col[2:].isdigit() and 1 <= int(col[2:]) <= 14]

    # Sort t-columns to ensure they're in order
    t_columns.sort(key=lambda x: int(x[2:]))

    # Create a StandardScaler
    scaler = StandardScaler()

    # Fit and transform t-columns
    scaled_t = scaler.fit_transform(df[t_columns])

    # Create a new dataframe with scaled values
    scaled_df = df.copy()
    scaled_df[t_columns] = scaled_t

    return scaled_df, scaler

In [27]:
scaled_data, scaler = scale_t_columns(final_df.copy())
cross_val_xgb(scaled_data.copy())

Prague_1 0.05566939313920342
Brno_1 0.06118257209802604
Prague_2 0.050448408044725127
Prague_3 0.055062098329582386
Munich_1 0.24586476763413825
Frankfurt_1 0.07223713415912815
Budapest_1 0.03876269117225859
0.08274672351100885


In [32]:
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Lasso
from sklearn.svm import SVR
from sklearn.preprocessing import StandardScaler
from sklearn.ensemble import StackingRegressor

In [33]:
xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', seed=42)
xgb_pipe = make_pipeline(xgb_regressor)

base_models = [
    ('xgb', xgb_pipe),
    ('gb', GradientBoostingRegressor(n_estimators=100, random_state=42)),
    ('lasso', Lasso(alpha=0.1))
]

# Define meta-model
meta_model = LinearRegression()

# Create the stacking regressor
stacking_regressor = StackingRegressor(
    estimators=base_models,
    final_estimator=meta_model,
    cv=5
)

cross_val_modelX(scaled_data, stacking_regressor)

Prague_1 0.04663671408963392
Brno_1 0.05882136039272083
Prague_2 0.05373028460984085
Prague_3 0.05040130663609471
Munich_1 0.18345056189653133
Frankfurt_1 0.07681861118616251
Budapest_1 0.03828067855270038
0.07259135962338349


## Find Stacking Regressor

In [43]:
import random
from sklearn.linear_model import (Ridge, Lasso, ElasticNet,
                                  SGDRegressor, HuberRegressor, TheilSenRegressor,
                                  RANSACRegressor, PoissonRegressor)
from sklearn.tree import DecisionTreeRegressor
from sklearn.ensemble import RandomForestRegressor, GradientBoostingRegressor
from sklearn.svm import SVR
from sklearn.neighbors import KNeighborsRegressor
from sklearn.neural_network import MLPRegressor
#from lightgbm import LGBMRegressor

def get_random_regressors():
    xgb_regressor = xgb.XGBRegressor(objective='reg:squarederror', seed=42)
    xgb_pipe = make_pipeline(xgb_regressor)
    regressors = [
        ('xgb', xgb_pipe),
        #('lightgbm', LGBMRegressor()),
        ('ridge', Ridge(alpha=1.0)),
        ('lasso', Lasso(alpha=0.1)),
        ('elastic_net', ElasticNet(alpha=0.1, l1_ratio=0.5)),
        ('sgd', SGDRegressor(max_iter=1000, tol=1e-3)),
        ('decision_tree', DecisionTreeRegressor()),
        ('random_forest', RandomForestRegressor()),
        ('gradient_boosting', GradientBoostingRegressor()),
        ('svr', SVR(kernel='rbf')),
        ('knn', KNeighborsRegressor()),
        #('mlp', MLPRegressor(max_iter=1000)),
        #('huber', HuberRegressor()),
        ('theil_sen', TheilSenRegressor()),
        ('ransac', RANSACRegressor()),
        ('poisson', PoissonRegressor())
    ]

    num_regressors = random.randint(3, min(10, len(regressors)))
    return random.sample(regressors, num_regressors)

In [44]:

for i in range(10):
    base_models = get_random_regressors()
    # Define meta-model
    meta_model = LinearRegression()
    print([i[0] for i in base_models])
    # Create the stacking regressor
    stacking_regressor = StackingRegressor(
        estimators=base_models,
        final_estimator=meta_model,
        cv=5
    )

    cross_val_modelX(scaled_data, stacking_regressor)

['sgd', 'ridge', 'ransac', 'svr', 'theil_sen', 'gradient_boosting', 'elastic_net']
Prague_1 0.044901074244948706
Brno_1 0.06361271174202558
Prague_2 0.05282265707874575
Prague_3 0.05366466218878503
Munich_1 0.15667416815864377
Frankfurt_1 0.09663522097180288
Budapest_1 0.040612942851633714
0.07270334817665505
['decision_tree', 'elastic_net', 'poisson', 'random_forest', 'theil_sen', 'lasso', 'sgd', 'svr', 'xgb']


/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions

Prague_1 0.047834367354589555


/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions

Brno_1 0.07671255090706906


...

Munich_1 0.18902404685572216


/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions

Frankfurt_1 0.09244568055381003


/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions/3.10.6/envs/lewagon/lib/python3.10/site-packages/sklearn/linear_model/_linear_loss.py:290: RuntimeWarning: invalid value encountered in matmul
  grad[:n_features] = X.T @ grad_pointwise + l2_reg_strength * weights
/home/marcel/.pyenv/versions

Budapest_1 0.05144315793706748
0.07825778136887242
['svr', 'xgb', 'decision_tree', 'gradient_boosting', 'lasso']
Prague_1 0.04591202543113432
Brno_1 0.05854299807133304
Prague_2 0.052737227111440146
Prague_3 0.050084034825279566
Munich_1 0.2038383917831048
Frankfurt_1 0.08318927897247234
Budapest_1 0.03802980877781283
0.07604768071036816
